In [ ]:
import tensorflow as tf
from tensorflow import nn
from tensorflow import layers

def cnn_net(train_data, V, W, w_bias):    
    # convolutional layer 1
    conv1=tf.layers.conv2d(train_data, filters=32,kernel_size=[5,5], padding='VALID', activation=tf.nn.relu)
    conv1=tf.layers.max_pooling2d(conv1, pool_size=[2,2], 
                             strides=2, padding='VALID')
    
    # convolutional layer 2
    conv2=tf.layers.conv2d(conv1, filters=64,kernel_size=[5,5], padding='VALID', activation=tf.nn.relu)
    conv2=tf.layers.max_pooling2d(conv2, pool_size=[2,2], 
                             strides=2, padding='VALID')
    
    # resize
    conv3=tf.transpose(tf.reshape(conv2,[-1,4*4*64]))

    # Hidden units
    h=tf.nn.relu(tf.matmul(V,conv3))

    # Output units
    z_pre_soft=tf.matmul(W,h)+w_bias
    z=tf.nn.softmax(z_pre_soft)
    z=tf.sigmoid(z_pre_soft)

    return z_pre_soft, z

print('Done')

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import nn
import random

# constants
# change window size
# check what to do if length<max_num_words
max_iter=10000
max_num_words=28
num_categ=10
input_vec_size=28
tot_input_vec_size=input_vec_size+1
num_hid_units=400
epsw=tf.constant(0.002,tf.float32)
stddevV=tf.cast(tf.truediv(1,num_filters_c1+1),tf.float32)
stddevW=tf.cast(tf.truediv(1,num_hid_units+1),tf.float32)

# creating placeholders train_data, train_labels, vali_data, vali_labels
# samples are along columns. Add biases (1) to data before use!
# Labels are already one-hot encoded 
# taking input_vec_size+1 to account for bias
train_data_pl=tf.placeholder(tf.float32,shape=(None,max_num_words,input_vec_size,1))
train_labels_pl=tf.placeholder(tf.int32,shape=(None,num_categ))
vali_labels_pl=tf.placeholder(tf.int32,shape=(None,))

# Define all variables
V=tf.Variable(tf.random_normal([num_hid_units,4*4*64],
                               mean=0.0,stddev=stddevV))
W=tf.Variable(tf.random_normal([num_categ,num_hid_units],
                               mean=0.0,stddev=stddevW))
w_bias=tf.Variable(tf.random_normal([num_categ,1],
                               mean=0.0,stddev=stddevW))

# 'Create' the main graph
z_pre_soft,z=cnn_net(train_data_pl,V,W,w_bias)

# Predicting
pred_labels=tf.argmax(z, axis = 0)

# loss
loss=tf.nn.softmax_cross_entropy_with_logits(labels = train_labels_pl, logits = tf.transpose(z_pre_soft))
# # loss=tf.nn.sigmoid_cross_entropy_with_logits(labels = tf.cast(train_labels_pl, tf.float32), 
# #                                              logits = tf.transpose(z_pre_soft))

# Session
sess=tf.InteractiveSession()

# Initializing
init=tf.global_variables_initializer()
sess.run(init)

# optimizing loss
optimizer=tf.train.GradientDescentOptimizer(epsw)
train_step=optimizer.minimize(loss)

minibatch=2

# running mini-batch SGD
for curr_iter in np.arange(max_iter):
    if curr_iter%5000==0:
        print('Curr iter is',curr_iter)
    
    # select appropriate mini-batch
    
    # generate random i
    i=random.randint(0,47999)
    if i<=47998:
        j=i+1
    else:
        j=0
    train_data_batch=traindata[[i,j],:,:,:]
    train_labels_batch=trainlabels[[i,j],:]

    sess.run(train_step,feed_dict={train_data_pl: train_data_batch,
                                   train_labels_pl: train_labels_batch})
#     i+=2

print('Done')

In [ ]:
# Accuracy
vali_labels_pl=tf.placeholder(tf.int32,shape=(None,))
vali_size_tf = tf.constant(vali_size)
accu = tf.truediv(vali_size_tf - tf.cast(tf.count_nonzero(
        tf.cast(pred_labels, tf.int32) - vali_labels_pl), tf.int32), vali_size_tf)

pred_labels_shape=tf.shape(pred_labels)
pred_labels_nzero=tf.count_nonzero(pred_labels)
vali_labels_shape=tf.shape(vali_labels_pl)

sess.run(accu, feed_dict = {train_data_pl: validata, vali_labels_pl: valilabels})